# Equity Examples
(c) Marek Ozana

Example usage of polars-bloomberg in Equity space

In [1]:
import altair as alt
import polars as pl

from polars_bloomberg import BQuery

## S&P Annual Price Returns

In [2]:
# get S&P annual price returns
bin_names: list = [
    "< -50",
    "-50 to -40",
    "-40 to -30",
    "-30 to -20",
    "-20 to -10",
    "-10 to 0",
    "0 to 10",
    "10 to 20",
    "20 to 30",
    "30 to 40",
    "40 to 50",
    "> 50",
]
bins = list(range(-50, 60, 10))

query = f"""
let(
    #rets = dropna(pct_diff(px_last(dates=range(-100Y, 0Y), per=Y)));
    #bins = bins(#rets, {bins}, bin_names={bin_names});
)
get(#rets, #bins)
for(['SPX Index'])
"""
with BQuery() as bq:
    df = bq.bql(query).combine()

In [3]:
# Create Chart
base = alt.Chart(df, title="S&P 500 Annual Price Returns").encode(
    x=alt.X("#bins:O", title="Annual Returns (%)").sort(bin_names),
    y=alt.Y("count()", title="Count").stack("zero"),
)
bars = base.mark_bar().encode(
    color=alt.Color("year(DATE):O").scale(scheme="viridis").legend(None)
)
txt = base.mark_text(dy=9).encode(
    text=alt.Text("year(DATE):O"),
)

(bars + txt).properties(width=600, height=400).configure_axisX(labelAngle=0)

alt.LayerChart(...)

## S&P earning Per Share Estimates

In [27]:
# Download data for multiple fiscal periods
with BQuery(timeout=50000) as con:
    res = []
    for n in range(1, 4):
        tbl = con.bql(f"""
            let(#eps=headline_eps_market(
                    fa_period_offset={n},
                    fa_period_type=A,
                    as_of_date=range('2009-12-31', TODAY(), frq=W)
                    );
            )
            get(#eps)
            for(['SPX Index'])
        """).combine()
        res.append(tbl)
data = pl.concat(res).drop_nulls().sort(by="AS_OF_DATE")
data.head(3)

ID,#eps,CURRENCY,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE
str,f64,str,date,date,date
"""SPX Index""",62.776665,"""USD""",2009-12-31,2009-12-31,2009-12-31
"""SPX Index""",79.121505,"""USD""",2009-12-31,2009-12-31,2010-12-31
"""SPX Index""",95.300733,"""USD""",2009-12-31,2009-12-31,2011-12-31


In [29]:
# Create chart
lines = (
    alt.Chart(data)
    .mark_line()
    .encode(
        x=alt.X("AS_OF_DATE:T").title(None),
        y=alt.Y("#eps:Q").scale(zero=False).title("EPS Estimate"),
        color=alt.Color("year(PERIOD_END_DATE):O")
        .scale(scheme="category20b")
        .title("Year"),
    )
)
# Plot text at the last point. !! Using argmax trick to get the last point. !!
last_point = lines.mark_point().encode(
    x=alt.X("max(AS_OF_DATE):T").title(None),
    y=alt.Y("#eps:Q").aggregate(argmax="AS_OF_DATE"),
)
last_bar = last_point.mark_rule(opacity=0.6).encode(y2=alt.datum(0))
text = last_point.mark_text(align="left", dx=3, opacity=0.6).encode(
    text=alt.Text("#eps:Q").aggregate(argmax="AS_OF_DATE").format("0.0f"),
)

(lines + last_point + last_bar + text).properties(
    width=600, height=400, title="S&P EPS Estimates"
)

alt.LayerChart(...)